In [112]:
import nest
import csv
import pickle
import h5py
import time
import numpy as np
import pandas as pd
import os
from collections import defaultdict
# import matplotlib.pyplot as plt
nest.ResetKernel()

In [113]:
class EINetwork:
    def __init__(self) -> None:
        self.unique_id = []
        self.neurons = {}
        self.connections = defaultdict(list)
        self.num_neurons = 0
        self.BasicWeight = 50
        
        # count of inh and exh synapse
        self.nI = 0
        self.nE = 0
        
        self.neuron_model = 'iaf_psc_alpha'
        
        # nest.CopyModel('stdp_synapse_hom', 'e_syn')
        # nest.CopyModel('static_synapse', 'i_syn')
    
    def process_and_cache_data_csv(
        self,
        csv_file_path='../connectome_materials/connections_no_threshold.csv',
        cache_file_path='../cache_file/csv_read_to_cache.pkl'
    ):
        if os.path.exists(cache_file_path):
            print('\n*** dataset already in cache!!! ***')
            # with open(cache_file_path, 'rb') as f:
            #     self.unique_id, self.connections = pickle.load(f)
            #     return self.unique_id, self.connections
            return
            
        print('\n*** Processing CSV file... ***')
        
        data = pd.read_csv(csv_file_path)
        data = data.drop_duplicates(subset=['pre_root_id', 'post_root_id'], keep='last')
        self.unique_id = list(set(data['pre_root_id']).union(set(data['post_root_id'])))
        print('\n*** Unique_id Completed ***')
        
        self.connections = [[pre, post, syn, nt] for pre, post, syn, nt in zip(data['pre_root_id'], data['post_root_id'], data['syn_count'], data['nt_type'])]
        
        print('\n*** Saving processed data to cache... ***')
        with open(cache_file_path, 'wb') as f:
            pickle.dump((self.unique_id, self.connections), f)
        
        # return self.unique_id, self.connections
        return
            
    
    def Create_and_Connect(self, cache_file='../cache_file/network.h5', cache_file_path='../cache_file/csv_read_to_cache.pkl'):
        nest.ResetKernel()
        print('\n*** kernel has been reset ***')

        if os.path.exists(cache_file):
            print('\n*** network has been stored previously!!! ***')
            return
        
        print('\n*** Loading dataset from cache... ***')
        with open(cache_file_path, 'rb') as f:
            self.unique_id, self.connections = pickle.load(f)

        print('\n*** Creating Neurons... ***')
        all_neurons = nest.Create(self.neuron_model, n=len(self.unique_id))
        
        print(f'\n*** {len(self.unique_id)} neurons have been created ***')
        
        print(f'\n*** Building neurons dictionary... ***')
        self.neurons = dict(zip(self.unique_id, all_neurons))
        
        print(f'\n*** Dictionary has been created ***')
        
        print(f'\n*** Creating Connections... ***')
        
        for [pre, post, syn, nt] in self.connections:
            if nt == 'GABA':
                weight = -1 * self.BasicWeight * syn
            else:
                weight = self.BasicWeight * syn
            
            nest.Connect(
                self.neurons[pre],
                self.neurons[post],
                'one_to_one',
                {
                    'synapse_model': 'stdp_synapse',
                    'weight': weight
                }
            )

        print(f'\n*** {len(self.connections)} connections have been created ***')

    def store_connections(self, cache_file='../cache_file/network.h5'):
        if os.path.exists(cache_file):
            print('\n*** network has been stored previously!!! ***')
            return

        # connections = nest.GetConnections()
        
        
        # print('\n*** storing the network... ***')
        # with pd.HDFStore('../cache_file/source.h5', mode='w') as store:
        #     source = connections.get(
        #     ('source'), output='pandas'
        # )
        #     store.append('source', source, index=False)
        #     target = connections.get(
        #     ('target'), output='pandas'
        # )
        #     store.append('target', target, index=False)
        #     weight = connections.get(
        #     ('weight'), output='pandas'
        # )
        #     store.append('weight', weight, index=False)

        # print('\n*** network has been stored ***')
        print('\n*** storing the network... ***')
    
        # 使用 chunking 来分批处理连接
        connections = nest.GetConnections()
        total_connections = len(connections)
        
        chunk_size = 30000
        with pd.HDFStore(cache_file, mode='w') as store:
            for start in range(0, total_connections, chunk_size):
                end = min(start + chunk_size, total_connections)
                chunk = connections[start:end]
                
                # 获取当前chunk的数据
                source = chunk.get('source', output='pandas')
                target = chunk.get('target', output='pandas')
                weight = chunk.get('weight', output='pandas')
                
                # 将数据追加到HDF5文件
                store.append('source', source, index=False)
                store.append('target', target, index=False)
                store.append('weight', weight, index=False)
            num_neurons = pd.DataFrame({'num_neurons': [len(self.unique_id)]})
            store.append('num_neurons', num_neurons, index=False)
        print('\n*** network has been stored ***')
        

    

    def restore_network(self, cache_file='../cache_file/network.h5', cache_file_path='../cache_file/csv_read_to_cache.pkl'):

        print('\n*** Loading file from cache... ***')
        with pd.HDFStore(cache_file, mode='r') as store:
            source = store['source']
            target = store['target']
            weight = store['weight']
            num_neurons = store['num_neurons']['num_neurons'].iloc[0]

        # with open(cache_file_path, 'rb') as f:
        #     self.unique_id, self.connections = pickle.load(f)

        print('\n*** Creating neurons... ***')
        all_neurons = nest.Create(self.neuron_model, num_neurons)

        print('\n*** Reconstructing network... ***')
        nest.Connect(
            source.values.flatten(),
            target.values.flatten(),
            'one_to_one',
            {
                'synapse_model': 'stdp_synapse',
                'weight': weight.values.flatten()
            }
        )

        print('\n*** network has been reconstructed!!! ***')
        
        
        
        

In [114]:
MyNETwork = EINetwork()
MyNETwork.process_and_cache_data_csv()
MyNETwork.Create_and_Connect()




*** dataset already in cache!!! ***

*** kernel has been reset ***

*** Loading dataset from cache... ***


UnpicklingError: invalid load key, 'v'.

In [ ]:

MyNETwork.store_connections()


*** network has been stored previously!!! ***


In [ ]:
nest.ResetKernel()

MyNETwork.restore_network()



*** Loading file from cache... ***

*** Creating neurons... ***

*** Reconstructing network... ***

*** network has been reconstructed!!! ***


In [ ]:
start = time.time()
MyNETwork = EINetwork()
nest.ResetKernel()
MyNETwork.restore_network()
end = time.time()
print(f'{end-start} s')


*** Loading file from cache... ***

*** Creating neurons... ***

*** Reconstructing network... ***

*** network has been reconstructed!!! ***
2.551696538925171 s
